In [4]:
#Importing all the necessarry modules
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from pyimagesearch.preprocessing import MeanPreprocessor
from pyimagesearch.preprocessing import ImageToArrayPreprocessor
from pyimagesearch.preprocessing import SimplePreprocessor
from pyimagesearch.io import HDF5DatasetGenerator
from pyimagesearch.nn.conv import DeeperGoogLeNet
from pyimagesearch.callbacks import EpochCheckpoint
from keras.utils import np_utils
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
import os
import json
import h5py

In [7]:
TRAIN_HDF5 = "/media/pavle/HDD_disk/deep-learning/food-classification/food-101/train.hdf5"
VAL_HDF5 = "/media/pavle/HDD_disk/deep-learning/food-classification/food-101/validate.hdf5"
TEST_HDF5 = "/media/pavle/HDD_disk/deep-learning/food-classification/food-101/test.hdf5"

In [8]:
iap = ImageToArrayPreprocessor()

batch_s = 32
epochs = 50
#The dataset has this many classes, yes.
classes = 101

#Define data augmentationn.
aug = ImageDataGenerator(rotation_range=20, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, fill_mode="nearest")

# initialize the training and validation dataset generators
trainGen = HDF5DatasetGenerator(TRAIN_HDF5, batch_s, aug=aug,
    preprocessors=[iap], classes=classes)
valGen = HDF5DatasetGenerator(VAL_HDF5, batch_s,
    preprocessors=[iap], classes=classes)
testGen = HDF5DatasetGenerator(TEST_HDF5, batch_s,
    preprocessors=[iap], classes=classes) 

In [4]:
model = DeeperGoogLeNet.build(width=128, height=128, depth=3, classes=classes, reg=0.0002)
opt = Adam(1e-3)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
lr = 0.05

2022-02-18 23:23:07.036980: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-18 23:23:07.067764: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-18 23:23:07.068299: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-18 23:23:07.069339: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
cb = [EpochCheckpoint("checkpoints", every=5, startAt=0)]

In [ ]:
H = model.fit_generator(
    trainGen.generator(),
    steps_per_epoch=trainGen.numImages // batch_s,
    validation_data=valGen.generator(),
    validation_steps=valGen.numImages // batch_s,
    epochs=epochs,
    max_queue_size=batch_s * 2,
    callbacks=cb,
    verbose=1)

/home/pavle/.local/lib/python3.8/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-02-18 23:23:09.215540: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50


2022-02-18 23:23:13.661786: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8204
2022-02-18 23:23:14.079760: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-02-18 23:23:15.300181: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.79GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-02-18 23:23:15.300258: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.79GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-02-18 23:23:16.116757: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 940

1775/1775 [==============================] - 791s 440ms/step - loss: 5.6488 - accuracy: 0.0782 - val_loss: 25.0457 - val_accuracy: 0.0572
Epoch 2/50
1775/1775 [==============================] - 795s 446ms/step - loss: 5.1221 - accuracy: 0.0868 - val_loss: 7.6093 - val_accuracy: 0.0841
Epoch 3/50
1775/1775 [==============================] - 823s 464ms/step - loss: 4.2731 - accuracy: 0.1405 - val_loss: 6.4433 - val_accuracy: 0.0449
Epoch 4/50
1775/1775 [==============================] - 837s 472ms/step - loss: 3.9051 - accuracy: 0.1859 - val_loss: 15.2871 - val_accuracy: 0.0151
Epoch 5/50
1775/1775 [==============================] - 820s 462ms/step - loss: 3.6625 - accuracy: 0.2327 - val_loss: 5.1374 - val_accuracy: 0.1350
Epoch 6/50
1775/1775 [==============================] - 839s 473ms/step - loss: 3.4604 - accuracy: 0.2753 - val_loss: 8.0299 - val_accuracy: 0.0751
Epoch 7/50
1775/1775 [==============================] - 823s 464ms/step - loss: 3.2892 - accuracy: 0.3104 - val_loss: 4.0

In [ ]:
# close the HDF5 datasets
trainGen.close()
valGen.close()

In [5]:
#Loading a checkpoint because the training crashed at some point.
model = load_model("./checkpoints/epochs_45.hdf5")

2022-02-19 11:12:48.667821: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-19 11:12:48.713068: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-19 11:12:48.714045: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-19 11:12:48.716113: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [15]:
f = h5py.File(TEST_HDF5, 'r')
labels = open("/media/pavle/HDD_disk/deep-learning/food-classification/food-101/meta/labels.txt", "r").readlines()
testY = np.array(list(f["labels"]))
testY = np_utils.to_categorical(LabelEncoder().fit_transform(testY), len(labels))

In [16]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testGen.generator(), steps=testGen.numImages // batch_s)

[INFO] evaluating network...


In [20]:
#Excluding 2 items from testY because the model.predict leaves them out for some reason it seems.
print(classification_report(testY.argmax(axis=1)[:-2], predictions.argmax(axis=1), target_names=labels))

                          precision    recall  f1-score   support

              Apple pie
       0.26      0.11      0.15       498
         Baby back ribs
       0.48      0.54      0.51       250
                Baklava
       0.33      0.63      0.43       250
         Beef carpaccio
       0.47      0.77      0.58       250
           Beef tartare
       0.51      0.60      0.55       250
             Beet salad
       0.42      0.58      0.49       250
               Beignets
       0.26      0.89      0.40       250
               Bibimbap
       0.52      0.81      0.64       250
          Bread pudding
       0.28      0.30      0.29       250
      Breakfast burrito
       0.35      0.15      0.21       250
             Bruschetta
       0.57      0.28      0.38       250
           Caesar salad
       0.62      0.65      0.64       250
                Cannoli
       0.36      0.28      0.31       250
          Caprese salad
       0.43      0.64      0.51       250
         

/home/pavle/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/pavle/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/pavle/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, epochs), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, epochs), H.history["val_loss"], label="val_loss")
plt.plot(np.ar  0ange(0,epochs), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0,epochs), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Acuracy")
plt.legend()
plt.show()